<div style="display: flex; align-items: center; justify-content: center; flex-wrap: wrap;">
    <div style="flex: 1; max-width: 400px; display: flex; justify-content: center;">
        <img src="https://i.ibb.co/JBPWVYR/Logo-Nova-IMS-Black.png" style="max-width: 50%; height: auto; margin-top: 50px; margin-bottom: 50px;margin-left: 6rem;">
    </div>
    <div style="flex: 2; text-align: center; margin-top: 20px;margin-left: 8rem;">
        <div style="font-size: 28px; font-weight: bold; line-height: 1.2;">
            <span style="color: #22c1c3;">DL Project |</span> <span style="color: #08529C;">Predicting Rare Species from Images using Deep Learning</span>
        </div>
        <div style="font-size: 17px; font-weight: bold; margin-top: 10px;">
            Spring Semester | 2024 - 2025
        </div>
        <div style="font-size: 17px; font-weight: bold;">
            Master in Data Science and Advanced Analytics
        </div>
        <div style="margin-top: 20px;">
            <div>André Silvestre, 20240502</div>
            <div>Diogo Duarte, 20240525</div>
            <div>Filipa Pereira, 20240509</div>
            <div>Maria Cruz, 20230760</div>
            <div>Umeima Mahomed, 20240543</div>
        </div>
        <div style="margin-top: 20px; font-weight: bold;">
            Group 37
        </div>
    </div>
</div>

<div style="background: linear-gradient(to right, #22c1c3, #27b1dd, #2d9cfd, #090979); 
            padding: 1px; color: white; border-radius: 500px; text-align: center;">
</div>

## **📚 Libraries Import**

In [1]:
# System imports
import os
import sys
import time
import datetime
from tqdm import tqdm
from typing_extensions import Self, Any      # For Python 3.10
# from typing import Self, Any               # For Python >3.11

from pathlib import Path

# Data manipulation imports
import numpy as np
import pandas as pd  
import warnings
warnings.filterwarnings("ignore")

# Data visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning imports
import tensorflow as tf
from keras.ops import add
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling, Lambda, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras import regularizers                                                                           # For L2 regularization
# import visualkeras

# Evaluation imports
from keras.metrics import CategoricalAccuracy, AUC, F1Score, Precision, Recall

# Other imports
from itertools import product

# Set the style of the visualization
pd.set_option('future.no_silent_downcasting', True)   # use int instead of float in DataFrame
pd.set_option("display.max_columns", None)            # display all columns

# Disable warnings (FutureWarning)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Set random seed for reproducibility
np.random.seed(2025)

2025-04-24 11:20:26.451683: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745490026.661430   16088 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745490026.704613   16088 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745490027.136181   16088 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745490027.136240   16088 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745490027.136243   16088 computation_placer.cc:177] computation placer alr

In [2]:
# Creates a SSL context that does not verify the server’s certificate - Needed for downloading pretrained models
# Source: https://precli.readthedocs.io/0.3.4/rules/python/stdlib/ssl_create_unverified_context.html
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
print("TensorFlow Version:", tf.__version__)
print("Is TensorFlow built with CUDA?", tf.test.is_built_with_cuda())
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("GPU Device Name:", tf.test.gpu_device_name())                                # (if error in Google Colab: Make sure your Hardware accelerator is set to GPU. 
                                                                                    # Runtime > Change runtime type > Hardware Accelerator)

TensorFlow Version: 2.19.0
Is TensorFlow built with CUDA? True
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU Device Name: /device:GPU:0


I0000 00:00:1745490032.284722   16088 gpu_device.cc:2019] Created device /device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [4]:
# Get build information from TensorFlow
build_info = tf.sysconfig.get_build_info()

print("TensorFlow version:", tf.__version__)
print("Python version:", sys.version)
print("CUDA version:", build_info.get("cuda_version", "Not available"))
print("cuDNN version:", build_info.get("cudnn_version", "Not available"))

TensorFlow version: 2.19.0
Python version: 3.10.12 (main, Feb  4 2025, 14:57:36) [GCC 11.4.0]
CUDA version: 12.5.1
cuDNN version: 9


In [5]:
# Import custom module for importing data, visualization, and utilities
import utilities

## **🧮 Import Databases**

#### **🟨 Google Collab**

In [ ]:
# # Run in Google Collab to download the dataset already splitted
# # Source: https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drivez
# # Download the file from Google Drive using wget
# !wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate \
#   "https://drive.usercontent.google.com/download?id=1dmr2cGxgM-kp1aXlmd9cQzVCkcl4JTFo&export=download" -O- | \
#   sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p' > /tmp/confirm.txt

# # Read the confirmation token from the temporary file
# with open('/tmp/confirm.txt', 'r') as f:
#     confirm_token = f.read().strip()

# # Download the file using the confirmation token and cookies
# !wget --load-cookies /tmp/cookies.txt \
#   "https://drive.usercontent.google.com/download?id=1dmr2cGxgM-kp1aXlmd9cQzVCkcl4JTFo&export=download&confirm={confirm_token}" \
#   -O /content/RareSpecies_Split.zip

# # Clean up temporary files
# !rm /tmp/cookies.txt /tmp/confirm.txt

# # List files in the /content directory to verify the download
# !ls -lh /content/

# # Unzip the downloaded file
# !unzip /content/RareSpecies_Split.zip -d /content/

# # List the unzipped files to verify
# !ls -lh /content/

In [ ]:
# # Run in Google Collab to download the dataset already splitted
# # Source: https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drivez
# # Download the file from Google Drive using wget
# !wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate \
#   "https://drive.usercontent.google.com/download?id=1Gfzm6mdhQ75cAzLQAK_gf-NFKk1KDRXo&export=download" -O- | \
#   sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p' > /tmp/confirm.txt

# # Read the confirmation token from the temporary file
# with open('/tmp/confirm.txt', 'r') as f:
#     confirm_token = f.read().strip()

# # Download the file using the confirmation token and cookies
# !wget --load-cookies /tmp/cookies.txt \
#   "https://drive.usercontent.google.com/download?id=1Gfzm6mdhQ75cAzLQAK_gf-NFKk1KDRXo&export=download&confirm={confirm_token}" \
#   -O /content/InnovativeApproach.zip

# # Clean up temporary files
# !rm /tmp/cookies.txt /tmp/confirm.txt

# # List files in the /content directory to verify the download
# !ls -lh /content/

# # Unzip the downloaded file
# !unzip /content/InnovativeApproach.zip -d /content/

# # List the unzipped files to verify
# !ls -lh /content/

In [6]:
# Define the path to the data
train_SMOTE_OnlyAnimal_dir = Path("data/InnovativeApproach/OnlyAnimal_train_DataAugmentationSMOTE")
val_OnlyAnimal_dir = Path("data/InnovativeApproach/OnlyAnimal_val")
test_OnlyAnimal_dir = Path("data/InnovativeApproach/OnlyAnimal_test")

# For Google Collab
# train_SMOTE_OnlyAnimal_dir = Path("/content/InnovativeApproach/OnlyAnimal_train_DataAugmentationSMOTE")
# val_OnlyAnimal_dir = Path("/content/InnovativeApproach/OnlyAnimal_val")
# test_OnlyAnimal_dir = Path("/content/InnovativeApproach/OnlyAnimal_test")

In [7]:
# Image Generators 
n_classes = 202                                     # Number of classes (we already know this based on previous notebook)
image_size = (224, 224)                             # Image size (224x224)
img_height, img_width = image_size                  # Image dimensions
batch_size = 64                                     # Batch size
input_shape = (img_height, img_width, 3)            # Input shape of the model
value_range = (0.0, 1.0)                            # Range of pixel values

In [9]:
# Get class names from directory
class_names = sorted(os.listdir(train_SMOTE_OnlyAnimal_dir))
class_indices = {name: i for i, name in enumerate(class_names)}

# Import the image dataset from the directory
from utilities import load_images_from_directory
train_datagen, val_datagen, test_datagen = load_images_from_directory(train_SMOTE_OnlyAnimal_dir, val_OnlyAnimal_dir, test_OnlyAnimal_dir,
                                                                      labels='inferred', label_mode='categorical',
                                                                      class_names=class_names, color_mode='rgb',
                                                                      batch_size=batch_size, image_size=image_size, seed=2025, 
                                                                      interpolation='bilinear', crop_to_aspect_ratio=False, pad_to_aspect_ratio=False)

print(f"\nLoaded: Train ({train_datagen.cardinality().numpy() * batch_size}), "
        f"Val ({val_datagen.cardinality().numpy() * batch_size}), "
        f"Test ({test_datagen.cardinality().numpy() * batch_size})")

Found 47874 files belonging to 202 classes.


I0000 00:00:1745490106.108634   16088 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Found 1020 files belonging to 202 classes.
Found 1025 files belonging to 202 classes.

Loaded: Train (47936), Val (1024), Test (1088)


In [ ]:
# Check the shape of the data (batch_size, img_width, img_height, 3)
for x, y in train_datagen.take(1):
    print("Train (SMOTE with OnlyAnimal) batch shape:", x.shape, y.shape)
for x, y in val_datagen.take(1):
    print("Val (OnlyAnimal) batch shape:", x.shape, y.shape)
for x, y in test_datagen.take(1):
    print("Test (OnlyAnimal) batch shape:", x.shape, y.shape)

Train (SMOTE with OnlyAnimal) batch shape: (64, 224, 224, 3) (64, 202)


2025-04-24 11:21:55.741781: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Val batch shape: (64, 224, 224, 3) (64, 202)


2025-04-24 11:21:59.902255: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Test batch shape: (64, 224, 224, 3) (64, 202)


# <a class='anchor' id='3'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right, #22c1c3, #27b1dd, #2d9cfd, #090979); 
            padding: 10px; color: white; border-radius: 300px; text-align: center;">
    <center><h1 style="margin-left: 140px;margin-top: 10px; margin-bottom: 4px; color: white;
                       font-size: 32px; font-family: 'Avenir Next LT Pro', sans-serif;">
        <b>10 | Innovative Approaches - Models with only Animals</b></h1></center>
</div>

<br><br>

## **🐢🌿 Import Classification Results**

In [11]:
# Load the DataFrame with classification results
df_train_results = pd.read_csv("./AnimalClassification/TrainAnimalClassification_Joined.csv")
df_val_results = pd.read_csv("./AnimalClassification/ValAnimalClassification_Joined.csv")
df_test_results = pd.read_csv("./AnimalClassification/TestAnimalClassification_Joined.csv")

# **🥇 Best Model**

In [ ]:
# Create directories for saving model checkpoints and evaluation logs
os.makedirs("./ModelCallbacks/10_ConvNeXtBaseFinalModels", exist_ok=True)      # exist_ok=True | Create directory if it doesn't exist
os.makedirs("./ModelsEvaluation/10_ConvNeXtBaseFinalModels", exist_ok=True)

<div class="alert alert-block alert-danger">

> **FALTA POR OS PARÂMETROS DO MELHOR MODELO + Mudar nome dos modelos na Evaluation**

</div>

In [ ]:
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.applications.convnext import preprocess_input

class RareSpeciesCNN_ConvNeXtBase_FinalModel(Model):
    """Custom CNN for rare species classification using ConvNeXtBase.
    
    Architecture: ConvNeXtBase 
    """
    def __init__(self, n_classes=202):
        super().__init__()                          # Call the parent class constructor

        # ----------------------------------------------------------------------------------------------------------------------------------------
        # Preprocess image for ConvNeXtBase
        # ResNet152V2 requires images to be preprocessed in a specific way (Source: https://keras.io/api/applications/convnext/#convnextbase-function)
        self.convnextbase_preprocess = Lambda(lambda x: preprocess_input(x), name='ConvNeXtBase_Preprocess')
        
        # Load ConvNeXtBase Pretrained Model
        self.convnextbase = ConvNeXtBase(include_top=False,         # Do not include the top classification layer (we will add our own because we have a different number of classes)
                                         classes=n_classes,         # Number of classes
                                         weights='imagenet')        # Use ImageNet weights
        self.convnextbase.trainable = False                         # Freeze the convolutional layers in the base model (transfer learning)
        # -----------------------------------------------------------------------------------------------------------------------------------------

        # --- Classification Head ---
        self.global_avg_pool = GlobalAveragePooling2D(name="Global_Average_Pooling")      # Global Average Pooling layer
        self.dense1 = Dense(128, name="Dense_Layer1")                                     # Smaller intermediate dense layer
        self.dropout = Dropout(0.5, name="Dropout_Layer")                                 # Dropout layer for regularization
        self.dense_output = Dense(n_classes, activation='softmax', name="Output_Layer")   # Output layer with softmax activation (for multi-class classification)

    def call(self, inputs, training=False):
        x = inputs                                     # Input layer

        # Preprocess image for ConvNeXtBase
        x = self.convnextbase_preprocess(x)              # Preprocess the image for ConvNeXtBase
        
        # Pass through ConvNeXtBase model
        x = self.convnextbase(x, training=training)      # Pass through the model (ConvNeXtBase)

        # Classification Head
        x = self.global_avg_pool(x)                      # Global Average Pooling
        x = self.dense1(x)                               # Dense layer
        x = self.dropout(x, training=training)           # Dropout layer
        outputs = self.dense_output(x)                   # Output layer
        return outputs

# Example Instantiation and Summary
model = RareSpeciesCNN_ConvNeXtBase_FinalModel(n_classes=n_classes)

# Build the model by providing an input shape
inputs = Input(shape=(img_width, img_height, 3))        # Input shape
_ = model.call(inputs)                                  # Call the model to build it
model.summary()                                         # Print the model summary

### 🐢🌿 **Train/Validation/Test `Only Animals`**

In [ ]:
# Compile model
optimizer = Adam(learning_rate=0.001, weight_decay=0.01, name="Optimizer")                                        # Adam for faster convergence
loss = CategoricalCrossentropy(name="Loss")                                                                       # Suitable for multi-class one-hot labels
metrics = [CategoricalAccuracy(name="accuracy"), Precision(name="precision"), Recall(name="recall"), F1Score(average="macro", name="f1_score"), AUC(name="auc")]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Callbacks
model_name = f"RareSpeciesCNN_ConvNeXtBase_FinalModels_{datetime.datetime.now().strftime('%Y%m%d')}" # Model name
callbacks = [ModelCheckpoint(f"./ModelCallbacks/10_ConvNeXtBaseFinalModels/checkpoint_{model_name}.keras", monitor="val_loss", save_best_only=True, verbose=0), 
             CSVLogger(f"./ModelCallbacks/10_ConvNeXtBaseFinalModels/metrics_{model_name}.csv"), 
             LearningRateScheduler(lambda epoch, lr: lr * 0.95), EarlyStopping(monitor='val_loss', patience=3, verbose=1),
             ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)] # Reduce learning rate if no improvement in validation loss

In [ ]:
# Train model
start_time = time.time()
history = model.fit(train_datagen, batch_size = batch_size, epochs=100, validation_data=val_datagen, callbacks=callbacks, verbose=1)
train_time = round(time.time() - start_time, 2)
print(f"\nTraining completed in \033[1m{train_time} seconds ({str(datetime.timedelta(seconds=train_time))} h)\033[0m).")

#### **🧪 Model Selection & 📏 Model Evaluation**

In [ ]:
# Evaluate model
from utilities import plot_metrics
plot_metrics(history, file_path=f"./ModelsEvaluation/10_ConvNeXtBaseFinalModels/10_TrainingValidationMetrics_{model_name}.png")

In [ ]:
# Evaluate on validation and test sets
train_results = {'accuracy': history.history['accuracy'][-1], 'precision': history.history['precision'][-1], 'recall': history.history['recall'][-1], 'f1_score': history.history['f1_score'][-1], 'auc': history.history['auc'][-1]}
val_results = model.evaluate(val_datagen, batch_size=batch_size, return_dict=True, verbose=1)
test_results = model.evaluate(test_datagen, batch_size=batch_size, return_dict=True, verbose=1)

In [ ]:
# Display results
from utilities import display_side_by_side, create_evaluation_dataframe
results_df = create_evaluation_dataframe(
    model_name="ConvNeXtBase - Final Model", variation="SMOTE (OnlyAnimals)",
    train_metrics=train_results, val_metrics=val_results, test_metrics=test_results, train_time=train_time,
    csv_save_path= f"./ModelsEvaluation/10_ConvNeXtBaseFinalModels/10_TrainingValidationMetrics_{model_name}.csv"
)
display_side_by_side(results_df, super_title="Model Evaluation Results")

In [ ]:
# Plot n right and n wrong predictions
from utilities import plot_predictions
plot_predictions(
    model=model,
    class_names=class_names,
    train_dir=train_SMOTE_OnlyAnimal_dir,
    test_data=test_datagen,
    num_images=10,
    file_path=f"./ModelsEvaluation/10_ConvNeXtBaseFinalModels/10_TestPredictions_{model_name}.png",
)

In [ ]:
# Plot confusion matrix
from utilities import plot_confusion_matrix

# Extract y_true from the dataset
y_true = []
for batch in test_datagen:
    # If label_mode is 'categorical', labels are one-hot encoded; convert to integers:
    images, labels = batch
    if labels.ndim > 1:
        y_true.extend(np.argmax(labels.numpy(), axis=1))
    else:
        y_true.extend(labels.numpy())
y_true = np.array(y_true)

# Get predictions from the model
y_pred = model.predict(test_datagen, batch_size=batch_size)
y_pred = np.argmax(y_pred, axis=1)

# Plot confusion matrix for test set
plot_confusion_matrix(
    y_true=y_true, y_pred=y_pred,
    title="Confusion Matrix | Best Model",
    file_path="./ModelsEvaluation/10_ConvNeXtBaseFinalModels/10_TestConfusionMatrix.png",
)

---

#### **🟨 Google Collab**

In [ ]:
# # Save Google Collab Workspace
# # Source: https://stackoverflow.com/questions/48774285/how-to-download-file-created-in-colaboratory-workspace

# # Zip the directories
# !zip -r /content/ModelCallbacks.zip /content/ModelCallbacks
# !zip -r /content/ModelsEvaluation.zip /content/ModelsEvaluation

# # Download the zip files
# from google.colab import files
# files.download("/content/ModelCallbacks.zip")
# files.download("/content/ModelsEvaluation.zip")

---

# **🔗 Bibliography/References**

**[[1]](https://huggingface.co/openai/clip-vit-base-patch16)** OpenAI. (2021). openai/clip-vit-base-patch16 · Hugging Face. Huggingface.co. https://huggingface.co/openai/clip-vit-base-patch16

**[[2]](https://arxiv.org/pdf/2201.03545)** Liu, Z., Mao, H., Wu, C.-Y., Feichtenhofer, C., Darrell, T., Xie, S., Facebook, A., & Research. (2022). A ConvNet for the 2020s. https://arxiv.org/pdf/2201.03545